In [133]:
!pip install wikipedia2vec
!pip install gensim

In [134]:
from wikipedia2vec import Wikipedia2Vec
import os
import time
import pandas as pd
import numpy as np
import pickle
import gensim

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## **다음은 colab과 drive를 연동하는 과정입니다.**

성공하려면 여러분의 '내 드라이브'에 '18-2 트랜드'가 추가되어 있어야 한다.

In [135]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
!ls -ltr  # find directory

total 8
drwxr-xr-x 2 root root 4096 Nov 29 18:21 sample_data
drwx------ 3 root root 4096 Dec  4 08:50 drive


## 다음은 예제 코드입니다. 정상적으로 작동하면 결과 뜰 것임.


### Pretrained data는 두 개로 나눠져 있다.

Word : 내용 전체 포함. 소문자로만 되어 있음.

Entity : 표제만 포함. 대문자 그대로임.

In [0]:
MODEL_FILE = '/content/drive/My Drive/18-2 트랜드/공유자료/enwiki/win10/enwiki_20180420_win10_100d.pkl'

In [0]:
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

In [0]:
wiki2vec.get_word_vector('ram')

memmap([-0.28108066, -0.24580207, -0.3453028 , -0.26693204,  0.07604776,
         0.07950985, -0.4013071 ,  0.371027  , -0.10209063, -0.16547424,
         0.35546365, -0.03499337,  0.4810935 , -0.0247711 ,  0.6654675 ,
         0.6868748 , -0.17947458, -0.25071502, -0.38239673, -0.07289735,
         0.21246287, -0.3406438 , -0.13159232,  0.64228356,  0.20363979,
        -0.59234893,  0.5206554 ,  0.4793145 , -0.11698707, -0.35107395,
         0.59233916,  0.15098831,  0.21566284,  0.06741711,  0.17410706,
         0.61831564,  0.32435963, -0.02891044, -0.11541634,  0.20510677,
         0.35892186, -0.12593259, -0.65592724,  0.06770121,  0.25496083,
         0.29104623,  0.20235395, -0.18535103, -0.4432048 ,  0.29753804,
        -0.39952484, -0.32598898,  0.1489867 ,  0.50611806,  0.09769119,
         0.36249286,  0.66147834, -0.18253979, -0.07603329,  0.07806999,
         0.17558034,  0.48491225, -0.25928882, -0.47785017,  0.23999836,
        -0.48740143,  0.15503086,  0.28547668,  0.3

In [0]:
wiki2vec.most_similar(wiki2vec.get_word('machine'), 20)

[(<Word machine>, 0.99999994),
 (<Word machines>, 0.7877456),
 (<Word slostin>, 0.7861282),
 (<Word einheitsmaschinengewehr>, 0.75466883),
 (<Word l37a1>, 0.7406934),
 (<Word m1919a2>, 0.73582214),
 (<Word m240c>, 0.73292005),
 (<Word l11a1>, 0.7307443),
 (<Word ginaca>, 0.7287393),
 (<Word l8a1>, 0.7260536),
 (<Word mg50>, 0.72534555),
 (<Word palletizer>, 0.7219498),
 (<Word sgmt>, 0.7218502),
 (<Word m1918m1>, 0.71929586),
 (<Word 315in>, 0.7185614),
 (<Word kleinflammenwerfer>, 0.7184659),
 (<Word mag58>, 0.7175655),
 (<Word defuzing>, 0.7171198),
 (<Word l43a1>, 0.7162625),
 (<Word l37a2>, 0.71545154)]

In [0]:
wiki2vec.get_entity_vector('Scarlett Johansson')

memmap([-7.3934019e-01, -4.9219817e-01, -4.2258736e-01,  5.4204553e-01,
        -1.1456601e-01, -6.9885385e-01, -4.5129403e-02, -2.2550362e-01,
        -4.8396614e-01,  4.3261728e-01,  7.7380174e-01, -2.7866521e-01,
        -2.3515283e-01,  8.3402467e-01,  1.1610613e+00,  6.0611492e-01,
         3.3689237e-01, -6.9169790e-01,  2.5748409e-02, -1.8423977e-01,
        -4.7181186e-01, -6.2870152e-02, -2.7133191e-01,  6.3593417e-01,
         1.3977311e+00,  5.1813495e-01,  4.8215788e-02, -1.2084860e+00,
         1.7581634e-01,  2.2923271e-01,  1.4231011e-01,  1.9371472e-01,
         4.2364649e-02, -5.5062354e-01,  4.3934953e-01,  7.7591199e-01,
         5.8910728e-01, -1.0891356e-01,  3.5113403e-01,  4.5360583e-01,
         1.2908849e+00,  1.9025780e+00, -3.7083137e-01,  2.9599072e-02,
        -2.7487081e-01,  4.3255150e-01,  1.6109006e-01, -1.8625103e-01,
         2.7524075e-01, -8.8827187e-01, -1.5277929e+00, -3.0613847e-03,
         4.0349716e-01, -2.7297637e-01,  1.4601653e+00,  7.64744

## 키워드와 wiki 매칭해보기

In [0]:
fname = '/content/drive/My Drive/18-2 트랜드/공유자료/파일/all_articles_df.p'

In [138]:
all_articles_df = pd.read_pickle(fname)
all_articles_df.head()

,idPrefix,id,pubDate,keywords,title,abstract
0,bd,13rRUwwslvh,2015-01-01,[],Welcome to the IEEE Transactions on Big Data,Presents an editorial introducting the inaugur...
1,bd,13rRUygT7c8,2015-01-01,[],Introduction to the IEEE Transactions on Big Data,Presents an introduction to the inaugural issu...
2,bd,13rRUwvT9j1,2015-01-01,"[Data Integration, Big Data, Data Mining, Feat...",Methodologies for Cross-Domain Data Fusion: An...,Traditional data mining usually deals with dat...
3,bd,13rRUxBJhxj,2015-01-01,"[Semantics, Internet, Electronic Publishing, S...",Embracing Information Explosion without Chokin...,The explosive popularity of microblogging serv...
0,bd,13rRUxAStUe,2015-07-01,"[Data Models, Computational Modeling, Big Data...",Petuum: A New Platform for Distributed Machine...,What is a systematic way to efficiently apply ...


In [0]:
all_articles_df['pubDate'] = pd.to_datetime(all_articles_df['pubDate'], format='%Y-%m-%d', errors='coerce')
all_articles_df = all_articles_df[~ all_articles_df['pubDate'].isnull()]
all_articles_from1990 = all_articles_df[all_articles_df.pubDate >= pd.to_datetime("1990-01-01")]

In [0]:
quarterly_keywords = all_articles_from1990[['pubDate', 'keywords']].groupby(pd.Grouper(key='pubDate', freq="Q")).agg(sum) # quarterly. http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases 
quarterly_keywords = quarterly_keywords[quarterly_keywords['keywords'] != False]
quarterly_keywords = quarterly_keywords[quarterly_keywords.keywords.str.len() != 0]


In [0]:
def remove_none_in_list(li):
    li = [x for x in li if x != '']
    return li

In [0]:
quarterly_keywords.keywords = quarterly_keywords.keywords.apply(remove_none_in_list)

In [143]:
quarterly_keywords.head()

,keywords
pubDate,
1995-03-31,"[Error Detection Codes, Error Correction Codes..."
1995-09-30,"[Radiosity, Hierarchical Techniques, Clusterin..."
1996-06-30,"[Programming Environments, Parallel Programmin..."
1996-09-30,"[Transport Protocols, Minimum Latency Transpor..."
1996-12-31,[Sequential Circuits]


In [0]:
def keywords_frequency(li):
    freq = {}
    
    for i in li:
        if i in freq.keys():
            freq[i] += 1
        else:
            freq[i] = 1  # 이건 머임?
    
    for k, v in freq.items():
        if v < 10 :
            freq[k] = -1
        
    freq2 = {}

    for k, v in freq.items():
        if v != -1:
            freq2[k] = v
    
    return freq2

In [0]:
quarterly_keywords['relativeFrequency'] = quarterly_keywords.keywords.apply(keywords_relative_frequency)

In [146]:
all_keys = quarterly_keywords.keywords.sum()

print(len(all_keys))
len(set(all_keys))

116870


38042

In [0]:
all_freq = keywords_frequency(all_keys)

In [148]:
len(all_freq.keys())

1543

In [0]:
unique_keys = set(all_keys)

In [0]:
# 키워드가 Wikipedia에 있는지
def wiki_match(unique_keys):
    ext_keys = []
  
    for a in unique_keys:
        cases = [a, a.capitalize(), a.lower(), a.upper(), a.replace(' ',''), 
                 a.replace(' ','').lower(), a.replace(' ','').capitalize(),
                 a.replace(' ','-'), a.replace(' ','_'), a.capitalize()[:-1],
                 a.lower()[:-1], a.upper()[:-1] ]
        a_split = a.split(' ')
        for i in range(0, len(a_split)) :
            cases.append( ''.join(a_split[:i+1]) + ' '.join(a_split[i+1:]) )
            cases.append( ' '.join(a_split[:i+1]) + ' '.join(a_split[i+1:]) )
            cases.append( ' '.join(a_split[:i+1]) + ''.join(a_split[i+1:]) )

        for i in cases:
            if wiki2vec.get_entity(i) != None :
                ext_keys.append(i)  # 같은 작업 두번해서 삭제함 (exist 변수)
                break
            elif wiki2vec.get_word(i) != None:
                ext_keys.append(i)
                break

    #중복되는 키 삭제      
    ext_keys = set(ext_keys)
    ext_keys = list(ext_keys)
  
  return ext_keys

In [151]:
len(unique_keys)

38042

In [0]:
ext_keys = wiki_match(unique_keys)

In [153]:
len(ext_keys)

10064

In [0]:
diff_keys = unique_keys - set(ext_keys)

In [0]:
len(diff_keys)

33885

In [0]:
c = 0
for i in diff_keys:
    if c < 5:
        print(i, end=', ')
        c += 1

Data Variability, Online Information Services, Cipertext Policy Attribute Based Encryption, Video Sharing, Total Transmission Power Minimization, 

## 우리꺼 키워드 벡터값 저장

In [0]:
wiki2vec #우리의 모델.
#우리는 100d의 10309 키워드에 대해서 클러스터링 할 것이다. (10309 = len(ext_keys))

In [0]:
word_vectors = []
for k in ext_keys :
    if wiki2vec.get_entity(k) != None :
        temp = []
        target = wiki2vec.get_entity_vector(k)
        for i in range(len(target)) :
            temp.append(target[i])
        word_vectors.append(temp)
        #word_vectors.append(wiki2vec.get_entity_vector(k))
    elif wiki2vec.get_word(k) != None :
        temp = []
        target = wiki2vec.get_word_vector(k)
        for i in range(len(target)) :
            temp.append(target[i])
        word_vectors.append(temp)
        #word_vectors.append(wiki2vec.get_word_vector(k))

In [0]:
len(word_vectors)

10064

In [0]:
word_vectors = np.array(word_vectors)

In [0]:
print(wiki2vec.get_entity_vector('Multilayer perceptron').shape)
print(word_vectors[0].shape)
type(wiki2vec.get_entity_vector('Multilayer perceptron'))
type(word_vectors[0])

(100,)
(100,)


numpy.ndarray

# K-mean
클러스터링은 비지도 학습 기법

클러스터링은 유사성 등 개념에 기초해 몇몇 그룹으로 분류하는 기법
클러스터링의 목적은 샘플(실수로 구성된 n차원의 벡터)을 내부적으로는 비슷하지만 외부적으로 공통 분모가 없는 여러 그룹으로 묶는 것
특정 차원의 범위가 다른 차원과 차이가 크면 클러스터링 하기 전에 스케일을 조정해야 한다.

1.최초 센트로이드(centroid)(중심점)로 k개의 벡터를 무작위로 선정한다.
2.각 샘플을 그 위치에서 가장 가까운 센트로이드에 할당한다.
3.센트로이드의 위치를 재계산한다.
4.센트로이드가 더 이상 움직이지 않을 때까지 2와 3을 반복한다.

In [0]:
import pandas as pd
import numpy as np
#from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from bs4 import BeautifulSoup
import re
import time

from nltk.corpus import stopwords
import nltk.data

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## K means 첫번째

In [0]:
# 단어 벡터에서 k-means를 실행하고 일부 클러스터를 찍어본다.
start = time.time() # 시작시간

# 클러스터의 크기 "k"를 어휘 크기의 1/100로 설정한다.
num_clusters = int(len(ext_keys)/100)
# K means 를 정의하고 학습시킨다.
kmeans = KMeans( n_clusters = num_clusters )
idx = kmeans.fit_predict( word_vectors )
#kmeans.fit(X_tsne)

end = time.time() # 끝난시간
elapsed = end - start # 걸린 시간
print("Time taken fortransforming : ", elapsed, "seconds.")

Time taken fortransforming :  11.102804183959961 seconds.


In [0]:
# 각 어휘 단어를 클러스터 번호에 매핑되게 word/Index 사전을 만든다.
idx = list(idx)
names = ext_keys
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}
#     word_centroid_map = dict(zip( model.wv.index2word, idx ))

# 첫번째 클러스터의 처음 10개를 출력
for cluster in range(0,10):
    # 클러스터 번호를 출력
    print("\nCluster {}".format(cluster))
    
    # 클러스터번호와 클러스터에 있는 단어를 찍는다.
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['Traffic Signal', 'Traffic speed', 'Cities', 'Urban Planning', 'Mass transport', 'City', 'Rail transportation', 'Bridge', 'Roads', 'Traffic management', 'Traffic flows', 'Traffic flow', 'Bus', 'Public Transportation', 'Transport capacity', 'Intelligent Transportation Systems', 'Overpasses', 'Light rail system', 'Infrastructure', 'Commuting', 'Road network', 'Highways', 'Road transportation', 'Traffic', 'Airports', 'Transportation Planning', 'Traffic Control', 'Bridges', 'Transportation system', 'Transportation', 'Urban areas', 'Traffic congestion', 'Tunnel', 'Functional classification']

Cluster 1
['Sparse matrices', 'Optimisation', 'Nonlinear programming', 'Numerical optimization', 'Linear complementarity problem', 'Objective function', 'Constraint satisfaction', 'Optimality', 'Optimal', 'Convex functions', 'Optimization problem', 'Sparse system', 'Convex programming', 'Convex Function', 'Semidefinite programming', 'Mixed integer linear programming', 'Linear programs', 'Be

In [0]:
len(word_centroid_map)
print(word_centroid_map)
print(ext_keys)

{'Magnetometers': 62, 'Virtualization': 33, 'Through silicon via': 7, 'Multiple Instance Learning': 6, 'Iris recognition': 61, 'Avispa': 50, 'Non Destructive Testing': 64, 'Magnetic shielding': 80, 'Parasitic capacitance': 51, 'Undirected graph': 49, 'Software agent': 88, 'Gc content': 53, 'Nonlinear dynamical systems': 82, 'Blocking techniques': 44, 'Auto scaling': 33, 'Nonnegative matrices': 42, 'Technological innovation': 59, 'IBM Research': 81, 'History': 46, 'Gender identification': 52, 'Ant colonies': 12, 'Personal communication service': 28, 'Dfl': 50, 'Diffusion rate': 80, 'Cognitive Network': 72, 'Multihop routing': 10, 'user': 29, 'Social Issues': 78, 'intergraph': 87, 'Poisson equation': 42, 'Hypergraphs': 49, 'Haptic perception': 68, 'Relays': 51, 'Internet Protocol': 30, 'Protection mechanisms': 29, 'Synthesizers': 52, 'Small molecules': 85, 'Rc circuit': 56, 'Fault tolerant systems': 47, 'Explicit Semantic Analysis': 98, 'ONN': 72, 'suspenser': 58, 'media': 58, 'Data sign

In [0]:
print(centroids[:10])
print(labels[:10])

NameError: ignored

## Word2Vec으로 벡터화 한 단어를 t-SNE 를 통해 시각화

In [0]:
# 참고 https://stackoverflow.com/questions/43776572/visualise-word2vec-generated-from-gensim
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import gensim 
import gensim.models as g

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [132]:
#model = g.Doc2Vec.load(word_vectors)

#vocab = list(model.wv.vocab)
X = word_vectors

print(len(X))
print(X[0][:10])
tsne = TSNE(n_components=2)

start = time.time() # 시작시간

#X_tsne = tsne.fit_transform(X[:100,:]) # 100개의 단어에 대해서만 시각화를 원할 경우
X_tsne = tsne.fit_transform(X)

end = time.time() # 끝난시간
elapsed = end - start # 걸린 시간
print("Time taken fortransforming : ", elapsed, "seconds.")

10064
[-1.9420843   1.1827909   1.4146155  -0.7481441   0.36114883 -1.123707
  0.15603663  0.5429964  -0.5426038  -1.0083525 ]
Time taken fortransforming :  339.64410614967346 seconds.


In [0]:
df = pd.DataFrame(X_tsne, index=ext_keys, columns=['x', 'y'])
df.shape

In [0]:
df.head(10)

## 키워드 시각화

In [0]:
# 단어, 점 함께 출력

fig = plt.figure() # new figure 생성
fig.set_size_inches(30, 15) # 화면 크기
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'], s = 15)

for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=10)
plt.show()